In [ ]:
import datetime

# Color codes for output 
RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
CYAN = "\033[36m"
MAGENTA = "\033[35m"
BOLD = "\033[1m"
RESET = "\033[0m"

menu = []   # All dishes will be stored here
stock = {   # Ingredient stock 
    "cheese": 3,
    "tomato": 12,
    "chicken": 8,
    "rice": 15,
    "lettuce": 5,
    "milk":7,
    "sugar":6
}

active_tables = set()  #  keep track of which tables are currently occupied
order_history = []     # Stores every completed order for reporting

#  PART 1: MENU MANAGEMENT 

def add_dish():
    print(BLUE + "--- Add New Dish ---" + RESET)
    name = input("Enter Dish Name: ")
    category = input("Enter Category (Appetizer/Main Course/Dessert/Beverage): ")

    try:
        # input validation
        price = float(input("Enter Price: "))
    except ValueError:
        print(RED + "Invalid price entered." + RESET)
        return

    # removing extra space
    ingredients_input = input("Enter ingredients separated by commas: ")
    ingredients = [i.strip() for i in ingredients_input.split(",")]

    dish = {
        "name": name,
        "category": category,
        "price": price,
        "ingredients": ingredients
    }

    menu.append(dish)
    print(GREEN + "Dish Added Successfully!\n" + RESET)


def view_menu():
    if not menu:
        print(RED + "Menu is empty.\n" + RESET)
        return

    print(BLUE + "\n" + "="*10 + " MENU " + "="*10 + RESET)
    for d in menu:
        print(MAGENTA + "Name:" + RESET, d["name"].title())
        print(CYAN + "Category:" + RESET, d["category"])
        print(GREEN + "Price:" + RESET, d["price"])
        print(YELLOW + "Ingredients:" + RESET, ", ".join(d["ingredients"]))
        print("-" * 30)


def search_dish():
    print(CYAN + "\n--- Search Dish ---" + RESET)
    print("1. Search by Name")
    print("2. Search by Category")
    ch = input("Choose: ")

    result = []

    # search options
    if ch == "1":
        name = input("Enter dish name: ")
        for d in menu:
            if d["name"].strip().lower() == name.strip().lower():
                result.append(d)

    elif ch == "2":
        cate = input("Enter category: ")
        for d in menu:
            if d["category"].strip().lower() == cate.strip().lower():
                result.append(d)

    else:
        print(RED + "Invalid choice.\n" + RESET)
        return

    if result:
        print(GREEN + f"\nFound {len(result)} result(s):" + RESET)
        for d in result:
            print("Name:", d["name"])
            print("Category:", d["category"])
            print("Price:", d["price"])
            print("Ingredients:", ", ".join(d["ingredients"]))
            print()
    else:
        print(RED + "No dish found.\n" + RESET)


def update_dish():
    print(CYAN + "--- Update Dish ---" + RESET)
    name_to_update = input("Enter dish name to update: ")
    flag = False

    for dish in menu:
        if dish["name"].lower() == name_to_update.lower():
            flag = True
            print("1. Update Price")
            print("2. Update Ingredients")
            choice = input("Choose: ")
            
            if choice == "1":
                #  price update
                new_price = float(input("Enter new price: "))
                dish["price"] = new_price
                print(GREEN + "Price updated.\n" + RESET)

            elif choice == "2":
                # updating full ingredient list
                new_ingr = input("Enter new ingredients (comma separated): ")
                dish["ingredients"] = [i.strip() for i in new_ingr.split(",")]
                print(GREEN + "Ingredients updated.\n" + RESET)

            else:
                print(RED + "Invalid choice.\n" + RESET)

            break

    if not flag:
        print(RED + "Dish not found.\n" + RESET)


def remove_dish():
    print(CYAN + "\n--- Remove Dish ---" + RESET)
    name = input("Enter dish name to remove: ")

    #  removing dish by matching name
    for d in menu:
        if d["name"].lower() == name.lower():
            menu.remove(d)
            print(GREEN + "Dish Removed.\n" + RESET)
            return

    print(RED + "Dish not found.\n" + RESET)


#  PART 2: ORDERING & BILLING 

def ordering():
    order = {}
    print(BLUE + "\n--- Place Order ---" + RESET)

    try:
        # making sure table number is valid
        table_number = int(input("Enter table number: "))
    except ValueError:
        print(RED + "Invalid Table Number." + RESET)
        return None

    # checking if table is already in use
    if table_number in active_tables:
        print(RED + f"Table {table_number} is already occupied! Cannot take new order." + RESET)
        return None
    else:
        active_tables.add(table_number)

    order["Table Number"] = table_number
    
    view_menu()  # showing menu before ordering

    while True:
        order_dish = input("Enter dish to order: ")

        found = False
        selected_dish = None
        
        # searching dish for ordering
        for d in menu:
            if d["name"].lower() == order_dish.lower():
                found = True
                selected_dish = d
                break

        if found:
            # warning management if stock is low
            for ingredient in selected_dish["ingredients"]:
                if stock.get(ingredient, 0) < 5:
                    print(RED + f"WARNING: Low stock on '{ingredient}' ({stock.get(ingredient)} left)!" + RESET)

            try:
                dish_quantity = int(input("How many plates?: "))
            except ValueError:
                print(RED + "Invalid quantity." + RESET)
                continue

            # updating order quantity
            if order_dish in order:
                order[order_dish] += dish_quantity
            else:
                order[order_dish] = dish_quantity
            
            print(GREEN + "Added to order!" + RESET)

            # deducting used ingredients
            for ingredient in selected_dish["ingredients"]:
                if ingredient in stock:
                    stock[ingredient] -= dish_quantity
                    if stock[ingredient] < 0:
                        stock[ingredient] = 0

        else:
            print(RED + "Dish is not available." + RESET)

        more = input("Do you want to order more? (yes/no): ")
        if more.lower() != "yes":
            break

    print(GREEN + f"Order taken for Table {table_number}" + RESET)
    return order


def generate_bill(order):
    if not order:
        return

    print(RESET + "\n" + "*" * 45)
    print(CYAN + BOLD + f"{'BILL RECEIPT':^45}" + RESET)
    print("*" * 45)

    table_num = order.get("Table Number")
    print(YELLOW + f"Table Number: {table_num}\n" + RESET)

    subtotal = 0
    discount_amount = 0
    current_month = datetime.datetime.now().month

    print(BLUE + f"{'Dish':<15} {'Qty':<5} {'Price':<10} {'Total'}" + RESET)
    print("-" * 45)

    for dish_name, qty in order.items():
        if dish_name == "Table Number":
            continue

        # searching dish again to get its price and category
        for d in menu:
            if d["name"].lower() == dish_name.lower():
                price = d["price"]
                total = 0
                promo_message = None 

                # buy 1 get 1 for appetizers
                if d["category"].lower() == "appetizer":
                    chargeable_qty = (qty // 2) + (qty % 2)
                    free_qty = qty - chargeable_qty
                    total = chargeable_qty * price
                    if free_qty > 0:
                        promo_message = GREEN + f"   (Promo: {free_qty} free)" + RESET

                else:
                    total = price * qty

                # december discount for desserts
                if d["category"].lower() == "dessert" and current_month == 12:
                    discount = total * 0.10
                    total -= discount
                    discount_amount += discount
                    promo_message = GREEN + "(Seasonal 10% Off Applied)" + RESET

                subtotal += total
                print(f"{dish_name:<15} {qty:<5} {price:<10} {total:.2f}")
                if promo_message:
                    print(promo_message)
                break

    tax = subtotal * 0.16
    grand_total = subtotal + tax

    print("-" * 45)
    print(f"Subtotal:      {subtotal:.2f}")
    if discount_amount > 0:
        print(GREEN + f"Discount Saved: {discount_amount:.2f}" + RESET)
    print(f"Tax (16%):     {tax:.2f}")
    print("-" * 45)
    print(GREEN + BOLD + f"GRAND TOTAL:   {grand_total:.2f}" + RESET)
    print("-" * 45 + "\n")

    # saving order into history for future reports
    saved_order = []
    for dish_name, qty in order.items():
        if dish_name != "Table Number":
            for d in menu:
                if d["name"].lower() == dish_name.lower():
                    saved_order.append({"dish": d, "qty": qty})
                    break
    order_history.append(saved_order)

    # table becomes free after bill
    if table_num in active_tables:
        active_tables.remove(table_num)


# PART 3: REPORTING & FORECASTING 

def sales_report():
    if not order_history:
        print("No sales recorded yet.\n")
        return

    dish_sales = {}
    revenue = 0

    # counting all dishes sold and total revenue
    for order in order_history:
        for item in order:
            dish = item["dish"]
            qty = item["qty"]
            revenue += dish["price"] * qty

            if dish["name"] not in dish_sales:
                dish_sales[dish["name"]] = 0
            dish_sales[dish["name"]] += qty

    print("\n--- SALES REPORT ---")
    print("Total Revenue:", revenue)
    best = max(dish_sales, key=dish_sales.get)
    print("Best Selling Dish:", best)
    least = min(dish_sales, key=dish_sales.get)
    print("Least Popular Dish:", least)
    print()


def inventory_report():
    print("\n--- INVENTORY REPORT ---")
    for ingredient, qty in stock.items():
        print(ingredient, ":", qty, "units")
        if qty < 5:
            # warning when stock is low
            print("RESTOCK NEEDED for:", ingredient)
    print()


def forecast_next_week():
    if not order_history:
        print("No past orders. Cannot forecast.\n")
        return

    predicted_usage = {}

    # estimating next week's ingredient usage based on past
    for order in order_history:
        for item in order:
            dish = item["dish"]
            qty = item["qty"]
            for ingredient in dish["ingredients"]:
                if ingredient not in predicted_usage:
                    predicted_usage[ingredient] = 0
                predicted_usage[ingredient] += qty

    print("\n--- FORECAST FOR NEXT WEEK ---")
    for ingredient, qty in predicted_usage.items():
        print(ingredient, "→ Need approx:", qty, "units next week")
    print()


#  MAIN MENU LOOP 

current_order = None

while True:
    print(BLUE + BOLD + "========= RESTAURANT MENU SYSTEM =========" + RESET)
    print(YELLOW + "1. Add New Dish" + RESET)
    print(YELLOW + "2. View Menu" + RESET)
    print(YELLOW + "3. Update Dish" + RESET)
    print(YELLOW + "4. Remove Dish" + RESET)
    print(YELLOW + "5. Search Dish" + RESET)
    print(YELLOW + "6. Ordering" + RESET)
    print(YELLOW + "7. Bill" + RESET)
    print(YELLOW + "8. Sales Report" + RESET)
    print(YELLOW + "9. Inventory Report" + RESET)
    print(YELLOW + "10. Forecast Next Week" + RESET)
    print(RED + "11. Exit" + RESET)

    option = input("Enter option: ")

    if option == "1":
        add_dish()

    elif option == "2":
        view_menu()

    elif option == "3":
        update_dish()

    elif option == "4":
        remove_dish()

    elif option == "5":
        search_dish()

    elif option == "6":
        current_order = ordering()

    elif option == "7":
        if current_order:
            generate_bill(current_order)
            current_order = None
        else:
            print(RED + "No order taken yet." + RESET)

    elif option == "8":
        sales_report()

    elif option == "9":
        inventory_report()

    elif option == "10":
        forecast_next_week()

    elif option == "11":
        print(MAGENTA + "Exiting program. Thank you!" + RESET)
        break

    else:
        print(RED + "Invalid Option." + RESET)


========= RESTAURANT MENU SYSTEM =========
1. Add New Dish
2. View Menu
3. Update Dish
4. Remove Dish
5. Search Dish
6. Ordering
7. Bill
8. Sales Report
9. Inventory Report
10. Forecast Next Week
11. Exit
